In [20]:
import os
directory = '/content/sample_data/CSE508_Winter2023_Dataset'
for filename in os.scandir(directory):
    if filename.is_file():
      with open(filename.path, "r") as file:
          lines = file.readlines()

      text = []
      text_started = False

      for line in lines:
        if "<TEXT>" in line:
          text_started = True
        elif "</TEXT>" in line:
          break
        elif text_started:
          text.append(line)


      title = []
      title_started = False

      for line in lines:
        if "<TITLE>" in line:
          title_started = True
        elif "</TITLE>" in line:
          break
        elif title_started:
          title.append(line)

      with open(filename.path, "w") as file:
        file.write("")
        file.write("".join(title))
        file.write("".join(text))

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
import os
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Define the list of stopwords to remove
stop_words = set(stopwords.words('english'))

# Loop through all the files in the directory
directory = '/content/sample_data/CSE508_Winter2023_Dataset'
count = 0
for filename in os.scandir(directory):
    if filename.is_file():
        # Open the file and read its contents
        with open(filename.path, "r") as file:
            text = file.read()

        # Step 1: Lowercase the text
        text = text.lower()

        # Step 2: Perform tokenization
        tokens = word_tokenize(text)

        # Step 3: Remove stopwords
        tokens = [token for token in tokens if not token in stop_words]

        # Step 4: Remove punctuations
        tokens = [token for token in tokens if not token in string.punctuation]

        # Step 5: Remove blank space tokens
        tokens = [token for token in tokens if not token.strip() == '']

        if count < 5:
            print(f"File: {filename}")
            print(f"Original text: {tokens[:5]}")
            print(f"After Step 1: {tokens[:5]}")
            tokens = word_tokenize(text)
            print(f"After Step 2: {tokens[:5]}")
            tokens = [token for token in tokens if not token in stop_words]
            print(f"After Step 3: {tokens[:5]}")
            tokens = [token for token in tokens if not token in string.punctuation]
            print(f"After Step 4: {tokens[:5]}")
            tokens = [token for token in tokens if not token.strip() == '']
            print(f"After Step 5: {tokens[:5]}\n")

        # Save the preprocessed text back to the same file
        with open(os.path.join(directory, filename), "w", encoding="utf-8") as file:
            file.write(" ".join(tokens))

        count += 1


File: <DirEntry 'cranfield0076'>
Original text: ['flight', 'measurement', 'wall', 'pressure', 'fluctuations']
After Step 1: ['flight', 'measurement', 'wall', 'pressure', 'fluctuations']
After Step 2: ['flight', 'measurement', 'of', 'wall', 'pressure']
After Step 3: ['flight', 'measurement', 'wall', 'pressure', 'fluctuations']
After Step 4: ['flight', 'measurement', 'wall', 'pressure', 'fluctuations']
After Step 5: ['flight', 'measurement', 'wall', 'pressure', 'fluctuations']

File: <DirEntry 'cranfield0621'>
Original text: ['latitude', 'diurnal', 'variations', 'air', 'densities']
After Step 1: ['latitude', 'diurnal', 'variations', 'air', 'densities']
After Step 2: ['latitude', 'and', 'diurnal', 'variations', 'of']
After Step 3: ['latitude', 'diurnal', 'variations', 'air', 'densities']
After Step 4: ['latitude', 'diurnal', 'variations', 'air', 'densities']
After Step 5: ['latitude', 'diurnal', 'variations', 'air', 'densities']

File: <DirEntry 'cranfield0908'>
Original text: ['random', 

In [23]:
# Set the directory path where the text files are stored
dir_path = "/content/sample_data/CSE508_Winter2023_Dataset"

# Get a list of all the text files in the directory
file_list = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]

# Read the contents of each text file into a list
docs = []
for file in file_list:
    with open(file, 'r') as f:
        text = f.read()
        docs.append(text)

# Create a CountVectorizer object and fit it to the list of documents
vec = CountVectorizer(binary=True)
X = vec.fit_transform(docs)

# Calculate the IDF values for each term
N = len(docs)
idf = {}
for term in vec.get_feature_names_out():
    df = sum(X[:, vec.vocabulary_[term]].toarray())
    inverseDocumentFrequency = math.log2((N) / (df + 1))
    idf[term] = inverseDocumentFrequency

In [9]:
import os
import pandas as pd
import math
from sklearn.feature_extraction.text import CountVectorizer

# Set the directory path where the text files are stored
dir_path = "/content/sample_data/CSE508_Winter2023_Dataset"

# Get a list of all the text files in the directory
file_list = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]

# Read the contents of each text file into a list
docs = []
for file in file_list:
    with open(file, 'r') as f:
        text = f.read()
        docs.append(text)

# Create a CountVectorizer object and fit it to the list of documents
vec = CountVectorizer(binary=True)
X = vec.fit_transform(docs)


# Create a dictionary to store the TF-IDF scores for each document
doc_tfidf = {}
for i, file in enumerate(file_list):
    tf = X[i].toarray()[0]
    tfidf = [tf[j] * idf[term] if tf[j] > 0 else 0 for j, term in enumerate(vec.get_feature_names_out())]
    doc_tfidf[os.path.basename(file)] = tfidf

# Create a dataframe from the dictionary of TF-IDF scores
df_tfidf_binary = pd.DataFrame.from_dict(doc_tfidf, orient='index', columns=vec.get_feature_names_out())
df_tfidf_binary.fillna(0, inplace=True)
df_tfidf_binary.head(5)


,00,000,0001,0005,000degree,000degreek,000efficient,000k,001,002,...,zero,zerolift,zeros,zeroth,zhukhovitskii,zone,zones,zoom,zuk,zurich
cranfield0076,0.0,4.750771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cranfield0621,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cranfield0908,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cranfield0203,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.342687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cranfield1319,0.0,4.750771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import CountVectorizer

# Set the directory path where the text files are stored
dir_path = "/content/sample_data/CSE508_Winter2023_Dataset"

# Get a list of all the text files in the directory
file_list = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]

# Read the contents of each text file into a list
docs = []
for file in file_list:
    with open(file, 'r') as f:
        text = f.read()
        docs.append(text)

# Create a CountVectorizer object and fit it to the list of documents
vec = CountVectorizer()
X = vec.fit_transform(docs)

# Calculate the TF-IDF for each term in each document
data = []
for i, file in enumerate(file_list):
    tf_term_doc = X[i].toarray()[0]
    tfidf = [tf_term_doc[j] * idf[term] if tf_term_doc[j] > 0 and term in idf else 0 for j, term in enumerate(vec.get_feature_names_out())]
    data.append(tfidf)

df_tfidf_raw = pd.DataFrame(data, columns=vec.get_feature_names_out(), index=[os.path.basename(f) for f in file_list])
df_tfidf_raw.head(20)


,00,000,0001,0005,000degree,000degreek,000efficient,000k,001,002,...,zero,zerolift,zeros,zeroth,zhukhovitskii,zone,zones,zoom,zuk,zurich
cranfield0076,0.0,9.501543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0621,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0908,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0203,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.685373,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield1319,0.0,4.750771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0732,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0957,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0119,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0540,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0960,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,13.501543,0.0,0.0,0.0,0.0


In [6]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import CountVectorizer

# Set the directory path where the text files are stored
dir_path = "/content/sample_data/CSE508_Winter2023_Dataset"

# Get a list of all the text files in the directory
file_list = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]

# Read the contents of each text file into a list
docs = []
for file in file_list:
    with open(file, 'r') as f:
        text = f.read()
        docs.append(text)

# Create a CountVectorizer object and fit it to the list of documents
vec = CountVectorizer()
X = vec.fit_transform(docs)

# Get the vocabulary list
vocab = vec.get_feature_names_out()

# Calculate the term frequency of each term in each document
tf_dict = {}
for i in range(X.shape[0]):
    tf = {}
    row = X.getrow(i)
    for j, val in zip(row.indices, row.data):
        tf[vocab[j]] = val / sum(row.data)
    tf_dict[i] = tf

# Convert the TF dictionary to a DataFrame
tf = pd.DataFrame(tf_dict).fillna(0)

# Calculate the TF-IDF for each term in each document
data = []
for i, file in enumerate(file_list):
    tf_term_doc = tf[i]
    tfidf = [tf_term_doc.get(term, 0) * idf[term] if tf_term_doc.get(term, 0) > 0 and term in idf else 0 for j, term in enumerate(vec.get_feature_names_out())]
    data.append(tfidf)

df_tfidf_term = pd.DataFrame(data, columns=vec.get_feature_names_out(), index=[os.path.basename(f) for f in file_list])
df_tfidf_term.head(20)


,00,000,0001,0005,000degree,000degreek,000efficient,000k,001,002,...,zero,zerolift,zeros,zeroth,zhukhovitskii,zone,zones,zoom,zuk,zurich
cranfield0076,0.0,0.077881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0621,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0908,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0203,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.163058,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield1319,0.0,0.032318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0732,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0957,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0119,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0540,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
cranfield0960,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.177652,0.0,0.0,0.0,0.0


In [4]:
import os
import pandas as pd
import math
from sklearn.feature_extraction.text import CountVectorizer

# Set the directory path where the text files are stored
dir_path = "/content/sample_data/CSE508_Winter2023_Dataset"

# Get a list of all the text files in the directory
file_list = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]

# Read the contents of each text file into a list
docs = []
for file in file_list:
    with open(file, 'r') as f:
        text = f.read()
        docs.append(text)

# Create a CountVectorizer object and fit it to the list of documents
vec = CountVectorizer()
X = vec.fit_transform(docs)

# Calculate the log term frequency for each term in each document
tf = pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out())
tf_log = tf.apply(lambda x: x.apply(lambda y: math.log2(1 + y)))

# Create a dataframe with the TF-IDF score of each term in each document
data = []
for i, file in enumerate(file_list):
    tf_log_doc = tf_log.iloc[i]
    tfidf = [tf_log_doc[j] * idf[term] if tf_log_doc[j] > 0 and term in idf else 0 for j, term in enumerate(vec.get_feature_names_out())]
    data.append(tfidf)

df_tfidf_log = pd.DataFrame(data, columns=vec.get_feature_names_out(), index=[os.path.basename(f) for f in file_list])
df_tfidf_log.head(20)


,00,000,0001,0005,000degree,000degreek,000efficient,000k,001,002,...,zero,zerolift,zeros,zeroth,zhukhovitskii,zone,zones,zoom,zuk,zurich
cranfield0076,0.0,7.529795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
cranfield0621,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
cranfield0908,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
cranfield0203,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.298033,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
cranfield1319,0.0,4.750771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
cranfield0732,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
cranfield0957,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
cranfield0119,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
cranfield0540,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
cranfield0960,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,10.69972,0.0,0.0,0.0,0.0


In [5]:
import os
import pandas as pd
import math
from sklearn.feature_extraction.text import CountVectorizer

# Set the directory path where the text files are stored
dir_path = "/content/sample_data/CSE508_Winter2023_Dataset"

# Get a list of all the text files in the directory
file_list = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]

# Read the contents of each text file into a list
docs = []
for file in file_list:
    with open(file, 'r') as f:
        text = f.read()
        docs.append(text)

# Create a CountVectorizer object and fit it to the list of documents
vec = CountVectorizer()
X = vec.fit_transform(docs)

tf_dn = pd.DataFrame(columns=vec.get_feature_names_out())
for i, file in enumerate(file_list):
    term_freq = dict(zip(vec.get_feature_names_out(), X.toarray()[i]))
    max_freq = max(term_freq.values())
    tf_doc = {}
    for word, freq in term_freq.items():
        tf_doc[word] = 0.5 + 0.5 * (freq / max_freq)
    tf_dn = tf_dn.append(tf_doc, ignore_index=True)

# Create a dataframe with the TF-IDF score of each term in each document
data = []
for i, file in enumerate(file_list):
    tf_log_doc = tf_dn.iloc[i]
    tfidf = [tf_log_doc[j] * idf[term] if tf_log_doc[j] > 0 and term in idf else 0 for j, term in enumerate(vec.get_feature_names_out())]
    data.append(tfidf)

df_tfidf_double_log = pd.DataFrame(data, columns=vec.get_feature_names_out(), index=[os.path.basename(f) for f in file_list])
df_tfidf_double_log.head(20)


<ipython-input-5-65942b0a2905>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tf_dn = tf_dn.append(tf_doc, ignore_index=True)
<ipython-input-5-65942b0a2905>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tf_dn = tf_dn.append(tf_doc, ignore_index=True)
<ipython-input-5-65942b0a2905>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tf_dn = tf_dn.append(tf_doc, ignore_index=True)
<ipython-input-5-65942b0a2905>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tf_dn = tf_dn.append(tf_doc, ignore_index=True)
<ipython-input-5-65942b0a2905>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fut

,00,000,0001,0005,000degree,000degreek,000efficient,000k,001,002,...,zero,zerolift,zeros,zeroth,zhukhovitskii,zone,zones,zoom,zuk,zurich
cranfield0076,3.640643,3.054067,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,1.671343,4.725606,4.725606,4.725606,4.725606,3.375386,4.433124,4.725606,4.725606,4.433124
cranfield0621,3.640643,2.375386,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,1.671343,4.725606,4.725606,4.725606,4.725606,3.375386,4.433124,4.725606,4.725606,4.433124
cranfield0908,3.640643,2.375386,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,1.671343,4.725606,4.725606,4.725606,4.725606,3.375386,4.433124,4.725606,4.725606,4.433124
cranfield0203,3.640643,2.375386,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,3.342687,4.725606,4.725606,4.725606,4.725606,3.375386,4.433124,4.725606,4.725606,4.433124
cranfield1319,3.640643,2.639317,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,1.671343,4.725606,4.725606,4.725606,4.725606,3.375386,4.433124,4.725606,4.725606,4.433124
cranfield0732,3.640643,2.375386,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,1.671343,4.725606,4.725606,4.725606,4.725606,3.375386,4.433124,4.725606,4.725606,4.433124
cranfield0957,3.640643,2.375386,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,1.671343,4.725606,4.725606,4.725606,4.725606,3.375386,4.433124,4.725606,4.725606,4.433124
cranfield0119,3.640643,2.375386,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,1.671343,4.725606,4.725606,4.725606,4.725606,3.375386,4.433124,4.725606,4.725606,4.433124
cranfield0540,3.640643,2.375386,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,1.671343,4.725606,4.725606,4.725606,4.725606,3.375386,4.433124,4.725606,4.725606,4.433124
cranfield0960,3.640643,2.375386,4.433124,4.725606,4.725606,4.433124,4.725606,4.725606,4.433124,4.725606,...,1.671343,4.725606,4.725606,4.725606,4.725606,4.500514,4.433124,4.725606,4.725606,4.433124


In [10]:

import pandas as pd
import string
import nltk
from nltk.corpus import stopwords

# assume we have already created the df_tfidf dataframe
# assume we have a query string stored in the variable query_str

query_str = input("Enter your query: ")

# text preprocessing
# convert to lowercase
query_str = query_str.lower()

# perform tokenization
tokens = nltk.word_tokenize(query_str)

# remove stopwords
stop_words = set(stopwords.words('english'))
tokens = [token for token in tokens if token not in stop_words]

# remove punctuations
tokens = [token for token in tokens if token not in string.punctuation]

# remove blank space tokens
tokens = [token for token in tokens if token.strip()]

# create a list of query terms
query_terms = tokens

# extract the relevant rows and columns from df_tfidf
print("tf idf by binay")
df_query = df_tfidf_binary.loc[:, query_terms]
# compute the score for each document
doc_scores = df_query.sum(axis=1)
# create a dictionary of document scores
doc_scores_dict = doc_scores.to_dict()
doc_scores_dict_sorted = sorted(doc_scores_dict.items(), key=lambda x:x[1],reverse=True)[:5]
print(doc_scores_dict_sorted)


print("tf idf by raw")
df_query = df_tfidf_raw.loc[:, query_terms]
# compute the score for each document
doc_scores = df_query.sum(axis=1)
# create a dictionary of document scores
doc_scores_dict = doc_scores.to_dict()
doc_scores_dict_sorted = sorted(doc_scores_dict.items(), key=lambda x:x[1],reverse=True)[:5]
print(doc_scores_dict_sorted)



print("tf idf by term freq")
df_query = df_tfidf_term.loc[:, query_terms]
# compute the score for each document
doc_scores = df_query.sum(axis=1)
# create a dictionary of document scores
doc_scores_dict = doc_scores.to_dict()
doc_scores_dict_sorted = sorted(doc_scores_dict.items(), key=lambda x:x[1],reverse=True)[:5]
print(doc_scores_dict_sorted)


print("tf idf by log")
df_query = df_tfidf_log.loc[:, query_terms]
# compute the score for each document
doc_scores = df_query.sum(axis=1)
# create a dictionary of document scores
doc_scores_dict = doc_scores.to_dict()
doc_scores_dict_sorted = sorted(doc_scores_dict.items(), key=lambda x:x[1],reverse=True)[:5]
print(doc_scores_dict_sorted)


print("tf idf by double_log")
df_query = df_tfidf_double_log.loc[:, query_terms]
# compute the score for each document
doc_scores = df_query.sum(axis=1)
# create a dictionary of document scores
doc_scores_dict = doc_scores.to_dict()
doc_scores_dict_sorted = sorted(doc_scores_dict.items(), key=lambda x:x[1],reverse=True)[:5]
print(doc_scores_dict_sorted)



Enter your query: investigation aerodynamics wing slipstream experimental study
tf idf by binay
[('cranfield0001', 23.43021004680389), ('cranfield0453', 17.428890407982507), ('cranfield1164', 15.489056626678828), ('cranfield1092', 14.311195365312752), ('cranfield0216', 13.942473058946447)]
tf idf by raw
[('cranfield1144', 66.81788989571287), ('cranfield0453', 58.980742404002456), ('cranfield0001', 57.62812355303386), ('cranfield0484', 52.211639107769045), ('cranfield1064', 47.438684939287214)]
tf idf by term freq
[('cranfield0001', 0.7294699183928337), ('cranfield0453', 0.46810113019049565), ('cranfield1064', 0.402022753722773), ('cranfield1090', 0.39421974182371045), ('cranfield1089', 0.3843642482781177)]
tf idf by log
[('cranfield0001', 37.994321234816965), ('cranfield0453', 33.147785233308895), ('cranfield1144', 30.269066501657925), ('cranfield1064', 24.52544433031658), ('cranfield0484', 23.90055854480635)]
tf idf by double_log
[('cranfield0001', 17.47791737870533), ('cranfield0453'

In [ ]:
import os
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Define the Jaccard coefficient function
def jaccard_coefficient(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

# Take query from the user and preprocess it
query = input("Enter your query: ")
query = query.lower()
query_tokens = nltk.word_tokenize(query)
query_tokens = [token for token in query_tokens if token not in stopwords.words('english')]
query_tokens = [token for token in query_tokens if token not in string.punctuation]
query_tokens = [token for token in query_tokens if token.strip()]

# Set the directory path where the text files are stored
dir_path = "/content/sample_data/CSE508_Winter2023_Dataset"

# Get a list of all the text files in the directory
file_list = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]

# Calculate the Jaccard coefficient between the query and each document in the directory
scores = []
for file in file_list:
    with open(file, 'r') as f:
        text = f.read()
        text = text.lower()
        doc_tokens = nltk.word_tokenize(text)
        doc_set = set(doc_tokens)
        score = jaccard_coefficient(set(query_tokens), doc_set)
        scores.append((os.path.basename(file), file, score))

# Sort the documents by their Jaccard coefficient scores in descending order
scores.sort(key=lambda x: x[2], reverse=True)

# Print out the top 5 documents with the highest Jaccard coefficient scores
for i in range(10):
    print(f"Document {i+1}: {scores[i][0]} ({scores[i][1]}), Jaccard coefficient score: {scores[i][2]}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter your query: experimental study of a wing in a propeller slipstream
Document 1: cranfield0001 (/content/sample_data/CSE/cranfield0001), Jaccard coefficient score: 0.0847457627118644
Document 2: cranfield0256 (/content/sample_data/CSE/cranfield0256), Jaccard coefficient score: 0.0625
Document 3: cranfield1045 (/content/sample_data/CSE/cranfield1045), Jaccard coefficient score: 0.058823529411764705
Document 4: cranfield1111 (/content/sample_data/CSE/cranfield1111), Jaccard coefficient score: 0.05714285714285714
Document 5: cranfield0507 (/content/sample_data/CSE/cranfield0507), Jaccard coefficient score: 0.05555555555555555
Document 6: cranfield1269 (/content/sample_data/CSE/cranfield1269), Jaccard coefficient score: 0.05263157894736842
Document 7: cranfield1094 (/content/sample_data/CSE/cranfield1094), Jaccard coefficient score: 0.05172413793103448
Document 8: cranfield0409 (/content/sample_data/CSE/cranfield0409), Jaccard coefficient score: 0.05128205128205128
Document 9: cranfiel